In [161]:
from dotenv import load_dotenv
load_dotenv()
import os
import pymysql

True

In [1]:
import gspread
from google.oauth2.service_account import Credentials

In [2]:
SCOPES = os.getenv("SCOPES").split(",")
CREDENTIALS_FILE = os.getenv("GOOGLE_SHEETS_CREDENTIALS")
DB_CONFIG = {
    "charset": "utf8mb4",
    "connect_timeout": 10,
    "cursorclass": pymysql.cursors.DictCursor,
    "db": os.getenv("DB_NAME"),
    "host": os.getenv("DB_HOST"),
    "password": os.getenv("DB_PASSWORD"),
    "read_timeout": 10,
    "port": int(os.getenv("DB_PORT")),
    "user": os.getenv("DB_USER"),
    "write_timeout": 10,
}

In [4]:
creds = Credentials.from_service_account_file(CREDENTIALS_FILE,scopes=SCOPES)

In [5]:
client = gspread.authorize(creds)

In [157]:
import gspread
from google.oauth2.service_account import Credentials
import pymysql
timeout = 10
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

In [158]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Loop through the tables and drop each one
for table in tables:
    table_name = table['Tables_in_defaultdb'] 
    if(table_name!="global_variables"): # Access the table name using the correct key
        drop_query = f"DROP TABLE IF EXISTS {table_name}"
        cursor.execute(drop_query)
        print(f"Dropped table: {table_name}")

# Commit the transaction (if necessary)
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

Dropped table: Superjoin_No_Dup
Dropped table: Superjoin_Sheet1
Dropped table: Superjoin_Sheet4
Dropped table: Superjoin_Sheet5
Dropped table: Superjoin_Sheet6
Dropped table: log_table


In [59]:
# SQL command to create the table to store global variables
create_table_query = """
CREATE TABLE IF NOT EXISTS global_variables (
    id INT AUTO_INCREMENT PRIMARY KEY,
    variable_name VARCHAR(255) NOT NULL,
    value INT NOT NULL
);
"""

# Execute the query to create the table
cursor.execute(create_table_query)

# Insert a single global variable called 'sheets' with value 1
insert_variable_query = """
INSERT INTO global_variables (variable_name, value)
VALUES ('sheets', 1)
ON DUPLICATE KEY UPDATE value = 1;
"""

# Execute the insert query
cursor.execute(insert_variable_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Table created and global variable inserted.")


Table created and global variable inserted.


In [64]:
query = "SHOW TABLES;"
cursor.execute(query)
res = cursor.fetchall()  # Fetches all rows from the executed query
res

[{'Tables_in_defaultdb': 'global_variables'}]

In [112]:
import gspread
from google.oauth2.service_account import Credentials
import pymysql


creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
client = gspread.authorize(creds)


timeout = 10
conn = conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# Step 3: Create log table for logging changes
# Step 3: Create log table for logging changes (including affected row ID)
create_log_table_query = """
CREATE TABLE IF NOT EXISTS log_table (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    table_name VARCHAR(255) NOT NULL,
    row_id INT NOT NULL -- Store the affected row's ID
);
"""
cursor.execute(create_log_table_query)
conn.commit()
print("Log table created or already exists.")



# Step 4: Set 'sheets' global variable to 0 at the start
update_sheets_query = "UPDATE global_variables SET value = 0 WHERE variable_name = 'sheets';"
cursor.execute(update_sheets_query)
conn.commit()
print("Updated 'sheets' value to 0.")

# Step 5: Prompt user for Google Sheets URL
sheets_url = "https://docs.google.com/spreadsheets/d/1wzA_QMaAHBBgNohNXnzuOJt5Jps8Dfx6H3NMA4zMB_Q/edit?gid=0#gid=0"
sheet = client.open_by_url(sheets_url)

# Step 6: Loop through all sheets
for sheets in sheet.worksheets():
    print(f"Processing sheet: {sheets.title}")
    
    # Get all rows and the header (first row)
    rows = sheets.get_all_values()
    
    if len(rows) == 0:
        print(f"Skipping empty sheet: {sheets.title}")
        continue
    
    # Step 7: Handle column names (first row as headers)
    values_list = rows[0]
    columns = [(f'`{col}`' if col else f'`Col{chr(65 + i)}`', 'VARCHAR(255)') for i, col in enumerate(values_list)]
    
    # Add ROW_NUMBER column to columns list
    columns.insert(0, ('`ROW_ID`', 'INT PRIMARY KEY'))
    
    # Generate the table name based on the sheet name
    table_name = f"`{sheet._properties['title']}_{sheets.title}`".replace(" ", "_")
    
    # Step 8: Create the table
    columns_str = ", ".join([f"{col_name} {col_type}" for col_name, col_type in columns])
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns_str}
    )
    '''
    cursor.execute(create_table_query)
    conn.commit()
    
    # Step 9: Insert data with ROW_NUMBER
    for i in range(1, len(rows)):  
        row_number = i  
        row_values = [row_number] + rows[i]
        
        # Prepare the placeholders for the VALUES clause
        placeholders = ", ".join(["%s"] * len(row_values))
        
        # Prepare the UPDATE clause
        update_clause = ", ".join([f"{col_name} = VALUES({col_name})" for col_name, _ in columns[1:]])
        
        # Construct the full INSERT query
        insert_query = f"""
        INSERT INTO {table_name} VALUES ({placeholders})
        ON DUPLICATE KEY UPDATE {update_clause}
        """
        
        cursor.execute(insert_query, row_values)
    
    conn.commit()
    print(f"Data inserted into {table_name}")

    # Step 10: Create triggers for INSERT, UPDATE, and DELETE
    # For INSERT trigger
    # For INSERT trigger
    insert_trigger_query = f"""
    CREATE TRIGGER after_insert_{table_name.replace('`', '')}
    AFTER INSERT ON {table_name}
    FOR EACH ROW
    BEGIN
        DECLARE sheets_value INT;
        SELECT value INTO sheets_value FROM global_variables WHERE variable_name = 'sheets';
        IF sheets_value = 1 THEN
            INSERT INTO log_table (table_name, row_id) VALUES ('{table_name.replace('`', '')}', NEW.ROW_ID);
        END IF;
    END;
    """

    
   # For UPDATE trigger
    update_trigger_query = f"""
    CREATE TRIGGER after_update_{table_name.replace('`', '')}
    AFTER UPDATE ON {table_name}
    FOR EACH ROW
    BEGIN
        DECLARE sheets_value INT;
        SELECT value INTO sheets_value FROM global_variables WHERE variable_name = 'sheets';
        IF sheets_value = 1 THEN
            INSERT INTO log_table (table_name, row_id) VALUES ('{table_name.replace('`', '')}', NEW.ROW_ID);
        END IF;
    END;
    """

    
    # For DELETE trigger
    delete_trigger_query = f"""
        CREATE TRIGGER after_delete_{table_name.replace('`', '')}
        AFTER DELETE ON {table_name}
        FOR EACH ROW
        BEGIN
            DECLARE sheets_value INT;
            SELECT value INTO sheets_value FROM global_variables WHERE variable_name = 'sheets';
            
            IF sheets_value = 1 THEN
                -- Insert the deleted row into the log_table
                INSERT INTO log_table (table_name, row_id) 
                VALUES ('{table_name.replace('`', '')}', OLD.ROW_ID);
                
                -- Reinsert the ROW_ID into the same table with other values as NULL
                INSERT INTO {table_name} (ROW_ID) 
                VALUES (OLD.ROW_ID);
            END IF;
        END;
        """


    
    cursor.execute(insert_trigger_query)
    cursor.execute(update_trigger_query)
    cursor.execute(delete_trigger_query)
    conn.commit()
    print(f"Triggers for INSERT, UPDATE, and DELETE created for {table_name}")

# Step 11: Set 'sheets' global variable to 1 at the end
update_sheets_query = "UPDATE global_variables SET value = 1 WHERE variable_name = 'sheets';"
cursor.execute(update_sheets_query)
conn.commit()
print("Updated 'sheets' value to 1.")

cursor.close()
conn.close()


Log table created or already exists.
Updated 'sheets' value to 0.
Processing sheet: Sheet1
Data inserted into `Superjoin_Sheet1`
Triggers for INSERT, UPDATE, and DELETE created for `Superjoin_Sheet1`
Processing sheet: No Dup
Data inserted into `Superjoin_No_Dup`
Triggers for INSERT, UPDATE, and DELETE created for `Superjoin_No_Dup`
Processing sheet: Sheet4
Data inserted into `Superjoin_Sheet4`
Triggers for INSERT, UPDATE, and DELETE created for `Superjoin_Sheet4`
Processing sheet: Sheet6
Data inserted into `Superjoin_Sheet6`
Triggers for INSERT, UPDATE, and DELETE created for `Superjoin_Sheet6`
Processing sheet: Sheet5
Data inserted into `Superjoin_Sheet5`
Triggers for INSERT, UPDATE, and DELETE created for `Superjoin_Sheet5`
Updated 'sheets' value to 1.


In [7]:
cursor.close()

True

In [6]:
conn.close()

In [89]:
import time
import gspread
import mysql.connector
from google.oauth2.service_account import Credentials
import re  # Regular expression to identify generic column names

creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# MySQL credentials

# Function to synchronize a specific row from MySQL table with the corresponding Google Sheet
def sync_table_with_sheet(table_name, row_id):
    try:
        # Get the corresponding Google Sheet
        sheet_name = "https://docs.google.com/spreadsheets/d/1wzA_QMaAHBBgNohNXnzuOJt5Jps8Dfx6H3NMA4zMB_Q/edit?gid=0#gid=0"  # Remove DB name part and replace _ with spaces
        sheet = client.open_by_url(sheet_name)

        # Create a new MySQL connection for this sync
        conn = conn = pymysql.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Retrieve the specific row from MySQL based on the row_id
        cursor.execute(f"SELECT * FROM `{table_name}` WHERE ROW_ID = %s", (row_id,))
        row = cursor.fetchone()
        
        if not row:
            print(f"No data found for ROW_NUMBER {row_id} in table {table_name}")
            return

        # Get the column names from the table, excluding the first column (ROW_NUMBER)
        cursor.execute(f"SHOW COLUMNS FROM `{table_name}`")
        columns = [column["Field"] for column in cursor.fetchall()[1:]]  # Skip the first column (ROW_NUMBER)

        # Check for generic column names like 'ColA', 'ColB', etc., and replace them with empty strings
        columns = ["" if re.match(r"Col[A-Z]", col) else col for col in columns]

        # Sync specific row with Google Sheets
        for worksheet in sheet.worksheets():
            if worksheet.title == table_name.split('_')[-1]:
                # Update the specific row in Google Sheets (skip the first element in the row which is ROW_NUMBER)
                row_values = list(row.values())[1:]  # Skip ROW_NUMBER
                worksheet.update(f"A{row_id}", [row_values])  # Update the specific row in the Google Sheet

                print(f"Synced row {row_id} of table {table_name} with Google Sheet {worksheet.title}")
                return
    except Exception as e:
        print(f"Error syncing table {table_name} and row {row_id} with Google Sheets: {str(e)}")
    finally:
        cursor.close()
        conn.close()

# Function to check the log table and sync tables
def monitor_log_table():
    try:
        while True:
            # Create a new MySQL connection inside the loop for every cycle
            conn = pymysql.connect(**DB_CONFIG)
            cursor = conn.cursor()

            # Query the log table for any logs
            cursor.execute("SELECT log_id, table_name, row_id FROM log_table")
            logs = cursor.fetchall()
            print(logs)
            if logs:

                for log in logs:
                    log_id, table_name, row_id = log["log_id"], log["table_name"], log["row_id"]

                    # Sync the corresponding row in the table
                    sync_table_with_sheet(table_name, row_id)

                    # Delete the processed log entry
                    cursor.execute(f"DELETE FROM log_table WHERE log_id = %s", (log_id,))
                    conn.commit()
                    print(f"Processed log for table {table_name}, row {row_id}, and removed log entry with ID {log_id}")

            # Close connection after processing logs
            cursor.close()
            conn.close()

            # Sleep for a few seconds before checking the log table again
            time.sleep(1)

    except KeyboardInterrupt:
        print("Stopping log monitoring.")
    except Exception as e:
        print(f"Error while monitoring log table: {str(e)}")

# Main function to start monitoring the log table
if __name__ == "__main__":
    monitor_log_table()


[{'log_id': 2, 'table_name': 'Superjoin_Sheet1', 'row_id': 2}]


C:\Users\DELL\AppData\Local\Temp\ipykernel_2120\1873068527.py:56: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(f"A{row_id}", [row_values])  # Update the specific row in the Google Sheet


Synced row 2 of table Superjoin_Sheet1 with Google Sheet Sheet1
Processed log for table Superjoin_Sheet1, row 2, and removed log entry with ID 2
()
Stopping log monitoring.


In [127]:
timeout = 10
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()
table_name =  "log_table"
row_id = 9
cursor.execute(f"SELECT * FROM `{table_name}`")
rows = cursor.fetchall()
print(rows)
cursor.close()
conn.close()

[{'log_id': 1, 'table_name': 'Superjoin_Sheet1', 'row_id': 2}]


In [160]:
timeout = 10
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()
table_name =  "Superjoin_Sheet1"
row_id = 2
cursor.execute(f"UPDATE {table_name} SET `UTR Number` = 'new_utr_value6' WHERE row_id = %s;",(row_id))
conn.commit()
cursor.close()
conn.close()

In [159]:
timeout = 10
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()
table_name = 'Superjoin_Sheet1'
cursor.execute(f"SELECT * FROM `{table_name}`")
l = cursor.fetchall()
rows = []
for i in l:
    li = [j for j in i.values()]
    rows.append(li)
for i in rows:
    print(i)
cursor.close()
conn.close()

[1, '432757234743', '103', '550', '', '']
[2, '346243623465', '103', '550', '', '']
[3, 'Test 2', '101', '400', None, None]
[4, '362963480599', '103', '400', '', '']
[5, '326715571530', '96', '1300', '', '']
[6, '326795965938', '96', '800', '', '']
[7, '327851688276', '82', '1200', '', '']
[8, '327929443807', '49', '400', '', '']
[9, '328296166011', '76', '600', '', '']
[10, '364412711414', '80', '400', '', '']
[11, '328648287317', '21', '850', '', '']
[12, '326363212116', '101', '550', '', '']
[13, '326340585320', '103', '400', '', '']
[14, '326487919228', '100', '550', '', '']
[15, '327736883087', '90', '1100', '', '']
[16, '327736921981', '90', '1100', '', '']
[17, '362910469569', '104', '400', '', '']
[18, '326340474835', '104', '400', '', '']
[19, '326362024483', '103', '1100', '', '']
[20, '326316843000', '101', '400', '', '']
[21, '326444399611', '98', '550', '', '']
[22, '326457555189', '98', '1650', '', '']
[23, '326450938461', '100', '800', '', '']
[24, '363010576481', '98', 

In [125]:
timeout = 10
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()
table_name = 'Superjoin_Sheet1'
cursor.execute(f"SELECT * FROM global_variables")
l = cursor.fetchall()
rows = []
for i in l:
    li = [j for j in i.values()]
    rows.append(li)
for i in rows:
    print(i)
cursor.close()
conn.close()

[1, 'sheets', 1]
